In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 모듈 임포트
import os
import json
import random
import csv
import subprocess

from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

import numpy as np
import cv2 as cv


In [ ]:
# 딕셔너리 & 배열 생성

# 1) path_lists.json 불러오기
SAVE_PATH = "/content/drive/MyDrive/ML/path_lists.json"
with open(SAVE_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

mp4_paths  = data["mp4_paths"]
json_paths = data["json_paths"]

# 2) base별 매핑
mp4_map  = {os.path.splitext(os.path.basename(p))[0]: p for p in mp4_paths}
json_map = {os.path.splitext(os.path.basename(p))[0]: p for p in json_paths}

# 3) 교집합된 base 정렬
bases = sorted(set(mp4_map) & set(json_map))

# 4) pairs를 튜플 리스트로 생성
pairs = [
    (b, mp4_map[b], json_map[b])
    for b in bases
]

In [ ]:
# 파일 경로 설정 및 폴더 생성

CSV_PATH = "/content/drive/MyDrive/ML/of_results.csv"
DONE_MARK = ".done"
OUT_DIR = "/content/markers/"
os.makedirs(os.path.dirname(CSV_PATH), exist_ok=True)
os.makedirs(OUT_DIR, exist_ok=True)

In [ ]:
# === Optical Flow 함수 ===
lk_params = dict(
    winSize=(15,15), maxLevel=2,
    criteria=(cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
)
GRID_SPACING = 20


In [ ]:
# 격자점 생성
def generate_grid_points(w, h, spacing):
    offset = spacing // 2
    pts = []
    for y in range(offset, h-offset, spacing):
        for x in range(offset, w-offset, spacing):
            pts.append([[x,y]])
    return np.array(pts, dtype=np.float32)

In [ ]:
# OF 계산 후 특징점 추출
def compute_of_stats_from_array(gray_frames):
    """
    gray_frames: NumPy array shape (45, H, W)
    returns: 18차원 리스트 [mean_cnt, std_cnt, mean_vx, std_vx, ... std_angle]
    """
    h, w = gray_frames.shape[1:]
    old_gray = gray_frames[0]
    p0 = generate_grid_points(w, h, GRID_SPACING)
    stats_list = []
    for i in range(1, gray_frames.shape[0]):
        frame_gray = gray_frames[i]
        p1, st, _ = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        if p1 is None:
            stats_list.append([0]*9)
            continue
        else:
            good = st.flatten()==1
            p1_flat = p1.reshape(-1, 2)
            p0_flat = p0.reshape(-1, 2)
            mv = p1_flat[good] - p0_flat[good]
            dx, dy = mv[:,0], mv[:,1]
            angs = np.degrees(np.arctan2(dy, dx))
            mask = (angs>-130)&(angs<-30)&(np.linalg.norm(mv,axis=1)>5)
            mv = mv[mask]
            if mv.size:
                ang2 = np.degrees(np.arctan2(mv[:,1], mv[:,0]))
                ma = ang2.mean(); df = np.abs(ang2-ma)
                df = np.where(df>180,360-df,df)
                mv = mv[df<40]
            cnt = mv.shape[0]
            if cnt:
                vx, vy = mv[:,0], mv[:,1]
                sp = np.linalg.norm(mv,axis=1)
                ang3 = np.degrees(np.arctan2(vy,vx))
                stats_list.append([
                    cnt,
                    float(vx.mean()), float(vy.mean()),
                    float(vx.std()),   float(vy.std()),
                    float(sp.mean()), float(sp.std()),
                    float(ang3.mean()), float(ang3.std())
                ])
            else:
                stats_list.append([0]*9)
        old_gray = frame_gray
        p0 = generate_grid_points(w, h, GRID_SPACING)

    arr = np.array(stats_list)
    summary = []
    for c in range(arr.shape[1]):
        summary += [ float(arr[:,c].mean()), float(arr[:,c].std()) ]
    return summary

In [ ]:
# ffmpeg 변환
def load_segment_as_gray(mp4_path, start_sec, duration_sec):
    """
    - start_sec: float, 시작 시각 (초)
    - duration_sec: float, 길이 (초)
    returns: np.ndarray of shape (frames, H, W), dtype=uint8
    """
    # FFmpeg 필터: fps=30 (60->30), scale=1080:720, format=gray
    cmd = [
      "ffmpeg",
      "-ss", f"{start_sec:.3f}",
      "-i", mp4_path,
      "-t", f"{duration_sec:.3f}",
      "-vf", "fps=30,scale=1080:720,format=gray",
      "-pix_fmt", "gray",   # 1채널 8bit
      "-f", "rawvideo",     # raw frame pipe
      "pipe:1"
    ]
    # 프로세스 생성
    p = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    # 예상 프레임 수
    num_frames = int(round(duration_sec * 30))
    H, W = 720, 1080
    expected_bytes = num_frames * H * W

    # 파이프에서 한 번에 읽기
    raw = b""
    while len(raw) < expected_bytes:
        chunk = p.stdout.read(expected_bytes - len(raw))
        if not chunk:
            break
        raw += chunk
    p.stdout.close()
    p.wait()

    if len(raw) != expected_bytes:
        raise RuntimeError(f"읽은 바이트가 예상과 다릅니다. {len(raw)}/{expected_bytes}")

    # NumPy 배열로 변환
    arr = np.frombuffer(raw, dtype=np.uint8)
    return arr.reshape(num_frames, H, W)

In [ ]:
def process_one(args):
    base, mp4_path, json_path = args

    # 1) JSON 로드
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    s60   = data["sensordata"].get("fall_start_frame", 0)
    e60   = data["sensordata"].get("fall_end_frame",   0)
    total = data["scene_info"].get("scene_length",    600)  # 60fps 프레임 수

    # 2) 윈도우 설정 (1.5초)
    window_sec = 1.5
    half_sec   = window_sec / 2.0

    # 3) 세그먼트 리스트 구성
    segments = []
    if s60 > 0 and e60 > 0:
        # 낙상: 중앙 구간
        mid60      = (s60 + e60) / 2.0
        mid_time   = mid60 / 60.0
        start_time = max(mid_time - half_sec, 0.0)
        segments.append((start_time, window_sec, 1, 0))
    else:
        # 비낙상: 랜덤 3개
        max_start = total / 60.0 - window_sec
        for sid in range(3):
            st = random.uniform(0.0, max_start)
            segments.append((st, window_sec, 0, sid))

    rows = []
    for start_time, duration, label, sid in segments:
        key    = (base, sid)
        marker = os.path.join(OUT_DIR, f"{base}_{sid}{DONE_MARK}")
        if key in processed or os.path.exists(marker):
            continue

        try:
            # 4) FFmpeg → 파이프 → NumPy 로 그레이 프레임 읽기
            gray_np = load_segment_as_gray(mp4_path, start_time, duration)
        except Exception as e:
            tqdm.write(f"⚠️ FFmpeg pipe failed for {base}_{sid}: {e}")
            continue

        # 5) OF 통계 계산
        stats = compute_of_stats_from_array(gray_np)
        rows.append([base, sid, label] + stats)

        # 6) 완료 마커
        open(marker, 'w').close()

    # 7) CSV 저장
    if rows:
        with open(CSV_PATH, 'a', newline='') as f:
            w = csv.writer(f)
            w.writerows(rows)

In [ ]:
if __name__ == "__main__":
    for args in tqdm(pairs, total=len(pairs), desc="Processing videos"):
        process_one(args)

Processing videos:  30%|███       | 2593/8592 [3:02:40<8:30:33,  5.11s/it]

⚠️ FFmpeg pipe failed for 01882_Y_A_N_C3_1: 읽은 바이트가 예상과 다릅니다. 34214400/34992000


Processing videos: 100%|██████████| 8592/8592 [10:12:33<00:00,  4.28s/it]
